# MRE: Federal CTC Expansion Reduces MA State Tax

This notebook demonstrates how federal CTC expansion can reduce Massachusetts state tax by triggering itemization.

In [1]:
from policyengine_us import Simulation
from policyengine_us.model_api import *

## Define a minimal tax unit

In [2]:
# Create a low-income family in MA with medical expenses
# The key is that they have very low income and medical expenses
situation = {
    "people": {
        "parent1": {
            "age": {2026: 45},
            "employment_income": {2026: 17_237},
            "medical_out_of_pocket_expenses": {2026: 3_339},
        },
        "parent2": {
            "age": {2026: 43},
        },
        "child1": {"age": {2026: 13}},
        "child2": {"age": {2026: 12}},
        "child3": {"age": {2026: 6}},
        "child4": {"age": {2026: 4}},
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent1", "parent2", "child1", "child2", "child3", "child4"],
        }
    },
    "households": {
        "household": {
            "members": ["parent1", "parent2", "child1", "child2", "child3", "child4"],
            "state_code": {2026: "MA"},
        }
    },
}

## Scenario 1: Without CTC Expansion

In [3]:
# Reform stack WITHOUT CTC (simulating Senate Finance Committee reforms)
reform_no_ctc = Reform.from_dict({
    # Tax rate changes
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.income.bracket.rates.4": {"2026-01-01": 0.28},
    # Standard deduction increase
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48_900},
    # Exemption elimination
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
}, country_id="us")

sim_no_ctc = Simulation(situation=situation, reform=reform_no_ctc)

print("WITHOUT CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_no_ctc.calculate('tax_unit_itemizes', 2026)[0])}")
print(f"  Federal tax (itemizing): ${sim_no_ctc.calculate('tax_liability_if_itemizing', 2026)[0]:,.2f}")
print(f"  Federal tax (standard): ${sim_no_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]:,.2f}")
print(f"  MA state tax: ${sim_no_ctc.calculate('ma_income_tax', 2026)[0]:,.2f}")
print(f"  MA Part B exemption: ${sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]:,.2f}")

# Show itemized deductions
print(f"\n  Itemized deductions breakdown:")
print(f"    Medical expense deduction: ${sim_no_ctc.calculate('medical_expense_deduction', 2026)[0]:,.2f}")
print(f"    State/local tax deduction: ${sim_no_ctc.calculate('salt_deduction', 2026)[0]:,.2f}")
print(f"    Total itemized: ${sim_no_ctc.calculate('itemized_taxable_income_deductions', 2026)[0]:,.2f}")
print(f"  Standard deduction: ${sim_no_ctc.calculate('standard_deduction', 2026)[0]:,.2f}")

WITHOUT CTC EXPANSION:


  Federal itemizes: False
  Federal tax (itemizing): $-9,892.20
  Federal tax (standard): $-9,892.20
  MA state tax: $-4,422.66
  MA Part B exemption: $12,800.00

  Itemized deductions breakdown:
    Medical expense deduction: $2,046.22


    State/local tax deduction: $641.85
    Total itemized: $2,688.07
  Standard deduction: $48,900.00


## Scenario 2: With CTC Expansion

In [4]:
# Add CTC expansion
reform_with_ctc = Reform.from_dict({
    # Same as above plus CTC
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.income.bracket.rates.4": {"2026-01-01": 0.28},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48_900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    # CTC expansion
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2_200},
}, country_id="us")

sim_with_ctc = Simulation(situation=situation, reform=reform_with_ctc)

print("WITH CTC EXPANSION:")
print(f"  Federal itemizes: {bool(sim_with_ctc.calculate('tax_unit_itemizes', 2026)[0])}")
print(f"  Federal tax (itemizing): ${sim_with_ctc.calculate('tax_liability_if_itemizing', 2026)[0]:,.2f}")
print(f"  Federal tax (standard): ${sim_with_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]:,.2f}")
print(f"  MA state tax: ${sim_with_ctc.calculate('ma_income_tax', 2026)[0]:,.2f}")
print(f"  MA Part B exemption: ${sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]:,.2f}")

# Show why they itemize
fed_tax_diff = sim_with_ctc.calculate('tax_liability_if_itemizing', 2026)[0] - sim_with_ctc.calculate('tax_liability_if_not_itemizing', 2026)[0]
print(f"\n  Federal tax difference (itemizing - standard): ${fed_tax_diff:,.2f}")

WITH CTC EXPANSION:


  Federal itemizes: False
  Federal tax (itemizing): $-9,892.20
  Federal tax (standard): $-9,892.20
  MA state tax: $-4,422.66
  MA Part B exemption: $12,800.00

  Federal tax difference (itemizing - standard): $0.00


## Analysis

In [5]:
# Calculate the impact
ma_tax_no_ctc = sim_no_ctc.calculate('ma_income_tax', 2026)[0]
ma_tax_with_ctc = sim_with_ctc.calculate('ma_income_tax', 2026)[0]
ma_tax_change = ma_tax_with_ctc - ma_tax_no_ctc

exemption_no_ctc = sim_no_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]
exemption_with_ctc = sim_with_ctc.calculate('ma_part_b_taxable_income_exemption', 2026)[0]
exemption_change = exemption_with_ctc - exemption_no_ctc

itemizes_no_ctc = bool(sim_no_ctc.calculate('tax_unit_itemizes', 2026)[0])
itemizes_with_ctc = bool(sim_with_ctc.calculate('tax_unit_itemizes', 2026)[0])

print("\nIMPACT OF FEDERAL CTC EXPANSION ON MA STATE TAX:")
print(f"  MA state tax change: ${ma_tax_change:,.2f}")
print(f"  MA Part B exemption change: ${exemption_change:,.2f}")
print(f"  Itemization change: {itemizes_no_ctc} → {itemizes_with_ctc}")

if itemizes_with_ctc and not itemizes_no_ctc:
    print(f"\nMECHANISM:")
    print(f"1. CTC expansion makes federal tax equal whether itemizing or not")
    print(f"2. PolicyEngine chooses to itemize when indifferent")
    print(f"3. MA allows itemizers to claim medical expense deduction")
    print(f"4. Medical expense deduction: ${sim_with_ctc.calculate('medical_expense_deduction', 2026)[0]:,.2f}")
    print(f"5. MA tax rate: 5%")
    print(f"6. Tax savings: ${exemption_change:,.2f} × 5% = ${exemption_change * 0.05:,.2f}")
elif abs(ma_tax_change) > 0.01:
    print(f"\nMA tax changed but itemization status didn't change.")
else:
    print("\nNo MA tax impact in this example.")


IMPACT OF FEDERAL CTC EXPANSION ON MA STATE TAX:
  MA state tax change: $0.00
  MA Part B exemption change: $0.00
  Itemization change: False → False

No MA tax impact in this example.
